### Required Libraries

In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

import h5py

from sklearn.preprocessing import scale

from keras.models import Sequential
from keras.layers import Dense, Activation, Conv2D, Reshape, Flatten, MaxPooling2D, Dropout
from keras.utils import np_utils

Using TensorFlow backend.


### Read and filter data

In [2]:
df_data = pd.read_csv('fer2013.csv')

df_labels = pd.read_csv('fer2013new.csv')
#changing output labels to categorical format
image_pixels_string = df_data[df_data.columns[1]]
image = np.zeros((df_data.shape[0], 48*48))

#forming image matrix of size 1, 48*48
for ix in range(image.shape[0]):
    temp = image_pixels_string[ix].split(' ')
    for iy in range(image.shape[1]):
        image[ix, iy] = int(temp[iy])

#usig sklearn to normalize each feature
image = scale(image, axis=0)

#forming training dataset and reshaping it to format accepted by input layer
X_train_num = (df_data[df_data['Usage']=='Training']).count()[0]
X_train = image[:X_train_num, :].reshape((X_train_num, 48, 48, 1))
y_train = df_labels[df_labels['Usage']=='Training'].iloc[:,2:]/10

#forming tcross validation dataset and reshaping it to format accepted by input layer
X_cv_num = (df_data[df_data['Usage']=='PublicTest']).count()[0]
X_cv = image[X_train_num:X_train_num+X_cv_num, :].reshape((X_cv_num, 48, 48, 1))
y_cv = df_labels[df_labels['Usage']=='PublicTest'].iloc[:,2:]/10

#forming ttesting dataset and reshaping it to format accepted by input layer
X_test_num = (df_data[df_data['Usage']=='PrivateTest']).count()[0]
X_test = image[X_train_num+X_cv_num:,:].reshape((X_test_num, 48, 48, 1))
y_test = df_labels[df_labels['Usage']=='PrivateTest'].iloc[:,2:]/10

### Model Creation

In [3]:
model = Sequential()

model.add(Conv2D(64, (3, 3), input_shape=(48, 48, 1), activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(64, activation='relu'))
model.add(Dense(10, activation='softmax'))

model.summary()
#keras.optimizers.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
model.compile(loss='categorical_crossentropy', optimizer='adam',
                  metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 46, 46, 64)        640       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 44, 44, 64)        36928     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 42, 42, 64)        36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 21, 21, 64)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 19, 19, 32)        18464     
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 17, 17, 32)        9248      
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 15, 15, 32)        9248      
__________

### Model fit and evaluation

In [4]:
hist = model.fit(X_train, y_train,
                     epochs=5,
                     shuffle=True,
                      batch_size=256,
                     validation_data=(X_cv, y_cv))

print('*****************************************')
score = model.evaluate(X_test, y_test, verbose=0)
print('* Testing loss : {}  *'.format(score[0]))
print('* Testing accuracy : {} *'.format(score[1]))
print('*****************************************')

Train on 28709 samples, validate on 3589 samples
Epoch 1/5
28709/28709 [==============================] - 503s 18ms/step - loss: 1.8704 - acc: 0.3501 - val_loss: 1.7258 - val_acc: 0.4132
Epoch 2/5
28709/28709 [==============================] - 501s 17ms/step - loss: 1.5852 - acc: 0.5450 - val_loss: 1.4288 - val_acc: 0.6166
Epoch 3/5
28709/28709 [==============================] - 489s 17ms/step - loss: 1.3986 - acc: 0.6335 - val_loss: 1.2922 - val_acc: 0.6626
Epoch 4/5
28709/28709 [==============================] - 492s 17ms/step - loss: 1.3173 - acc: 0.6710 - val_loss: 1.2162 - val_acc: 0.6877
Epoch 5/5
28709/28709 [==============================] - 492s 17ms/step - loss: 1.2489 - acc: 0.6991 - val_loss: 1.1778 - val_acc: 0.7152
*****************************************
* Testing loss : 1.2342256356428247  *
* Testing accuracy : 0.6968514906700756 *
*****************************************


### Saving model for prediction

In [5]:
model.save('live_emotion_recognition_model.h5')  # creates a HDF5 file 'my_model.h5'
fhandle = h5py.File('live_emotion_recognition_model.h5', 'r+')
del fhandle['optimizer_weights']
fhandle.close()